## 1. Load the libraries for calculation

In [1]:
# Import the modules of interest
import pandas as pd
import geopandas as gpd
import ee
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [2]:
# Intialize the ee API connection
ee.Initialize()

## 2. Prepare the composite for calculation

In [3]:
# Define the vectors of predictors
predictorVector = ['Aridity_Index',
                   'CHELSA_Annual_Mean_Temperature',
                   'CHELSA_Annual_Precipitation',
                   'CHELSA_Isothermality',
                   'CHELSA_Max_Temperature_of_Warmest_Month',
                   'CHELSA_Mean_Diurnal_Range',
                   'CHELSA_Mean_Temperature_of_Coldest_Quarter',
                   'CHELSA_Mean_Temperature_of_Driest_Quarter',
                   'CHELSA_Mean_Temperature_of_Warmest_Quarter',
                   'CHELSA_Mean_Temperature_of_Wettest_Quarter',
                   'CHELSA_Min_Temperature_of_Coldest_Month',
                   'CHELSA_Precipitation_Seasonality',
                   'CHELSA_Precipitation_of_Coldest_Quarter',
                   'CHELSA_Precipitation_of_Driest_Month',
                   'CHELSA_Precipitation_of_Driest_Quarter',
                   'CHELSA_Precipitation_of_Warmest_Quarter',
                   'CHELSA_Precipitation_of_Wettest_Month',
                   'CHELSA_Precipitation_of_Wettest_Quarter',
                   'CHELSA_Temperature_Annual_Range',
                   'CHELSA_Temperature_Seasonality',
                   'Depth_to_Water_Table',
                   'EarthEnvTopoMed_Eastness',
                   'EarthEnvTopoMed_Elevation',
                   'EarthEnvTopoMed_Northness',
                   'EarthEnvTopoMed_ProfileCurvature',
                   'EarthEnvTopoMed_Roughness',
                   'EarthEnvTopoMed_Slope',
                   'SG_Absolute_depth_to_bedrock',
                   'WorldClim2_SolarRadiation_AnnualMean',
                   'WorldClim2_WindSpeed_AnnualMean',
                   'EarthEnvCloudCover_MODCF_interannualSD',
                   'EarthEnvCloudCover_MODCF_intraannualSD',
                   'EarthEnvCloudCover_MODCF_meanannual',
                   'EarthEnvTopoMed_AspectCosine',
                   'EarthEnvTopoMed_AspectSine',
                   'LandCoverClass_Cultivated_and_Managed_Vegetation',
                   'Human_Disturbance',
                   'LandCoverClass_Urban_Builtup',
                   'Human_Development_Percentage',
                   'SG_Clay_Content_0_100cm',
                   'SG_Coarse_fragments_0_100cm',
                   'SG_Sand_Content_0_100cm',
                   'SG_Silt_Content_0_100cm',
                   'SG_Soil_pH_H2O_0_100cm',
                   'WDPA',
                   'cropland',
                   'grazing',
                   'pasture',
                   'rangeland']

In [4]:
# Load the composite for root shoot ratio analysis
compositeImg = ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImg = compositeImg.select(predictorVector)
# define the covariates to use
covarsToUse = compositeImg.bandNames()
print(colored('Number of covariates:\n', 'blue', attrs=['bold']),covarsToUse.getInfo())

Number of covariates:
 ['Aridity_Index', 'CHELSA_Annual_Mean_Temperature', 'CHELSA_Annual_Precipitation', 'CHELSA_Isothermality', 'CHELSA_Max_Temperature_of_Warmest_Month', 'CHELSA_Mean_Diurnal_Range', 'CHELSA_Mean_Temperature_of_Coldest_Quarter', 'CHELSA_Mean_Temperature_of_Driest_Quarter', 'CHELSA_Mean_Temperature_of_Warmest_Quarter', 'CHELSA_Mean_Temperature_of_Wettest_Quarter', 'CHELSA_Min_Temperature_of_Coldest_Month', 'CHELSA_Precipitation_Seasonality', 'CHELSA_Precipitation_of_Coldest_Quarter', 'CHELSA_Precipitation_of_Driest_Month', 'CHELSA_Precipitation_of_Driest_Quarter', 'CHELSA_Precipitation_of_Warmest_Quarter', 'CHELSA_Precipitation_of_Wettest_Month', 'CHELSA_Precipitation_of_Wettest_Quarter', 'CHELSA_Temperature_Annual_Range', 'CHELSA_Temperature_Seasonality', 'Depth_to_Water_Table', 'EarthEnvTopoMed_Eastness', 'EarthEnvTopoMed_Elevation', 'EarthEnvTopoMed_Northness', 'EarthEnvTopoMed_ProfileCurvature', 'EarthEnvTopoMed_Roughness', 'EarthEnvTopoMed_Slope', 'SG_Absolute_de

## 3. Prepare train data 

### 3.1 Load and check the train data

In [5]:
# load the train data with covariates
leafHabitDataRaw = ee.FeatureCollection('users/haozhima95/forest_pheno/sub_logitmatrix_seed10')
# sample the covariates
# rawSampled = compositeImg.sampleRegions(leafHabitDataRaw,geometries = True)
# add a column in the feature collection and do a random subsample
rawSampledWithRandom = leafHabitDataRaw.randomColumn('random', 1000)
# get the size of this feature collection
totalSize = rawSampledWithRandom.size().getInfo()
print(colored('Number of observations:', 'blue', attrs=['bold']),totalSize)
print(colored('Property names:/n', 'blue', attrs=['bold']),rawSampledWithRandom.first().propertyNames().getInfo())
varToModel = 'leaf_habit'

Number of observations: 4261
Property names:/n ['random', 'year', 'systemindex', 'CHELSA_Precipitation_of_Driest_Quarter', 'CHELSA_Mean_Temperature_of_Wettest_Quarter', 'LandCoverClass_Urban_Builtup', 'SG_Silt_Content_0_100cm', 'EarthEnvTopoMed_AspectSine', 'rangeland', 'EarthEnvTopoMed_Slope', 'SG_Absolute_depth_to_bedrock', 'SG_Clay_Content_0_100cm', 'cropland', 'CHELSA_Temperature_Seasonality', 'EarthEnvCloudCover_MODCF_meanannual', 'CHELSA_Annual_Mean_Temperature', 'EarthEnvCloudCover_MODCF_interannualSD', 'LandCoverClass_Cultivated_and_Managed_Vegetation', 'EarthEnvTopoMed_Elevation', 'CHELSA_Precipitation_of_Wettest_Month', 'CHELSA_Precipitation_of_Wettest_Quarter', 'index', 'Aridity_Index', 'WorldClim2_WindSpeed_AnnualMean', 'CHELSA_Precipitation_Seasonality', 'SG_Soil_pH_H2O_0_100cm', 'info_coverage', 'EarthEnvTopoMed_Roughness', 'CHELSA_Temperature_Annual_Range', 'Human_Development_Percentage', 'SG_Sand_Content_0_100cm', 'CHELSA_Annual_Precipitation', 'no_plots', 'CHELSA_Preci

## 4. Spatial cross validation for Leaf habit

### 4.1 Run spatial cross validation

In [6]:
# Define list contains the buffer sizes to test
buffer_sizes = [10,100000,150000,200000,250000,500000,1000000] # 150km,200km,250km

# define the core function for spatial cross validation
#  Blocked Leave One Out cross-validation function:
def BLOOcv(f):
    rep = f.get('rep')
    # Test feature
    testFC = ee.FeatureCollection(f)

    # Training set: all samples not within geometry of test feature
    trainFC = rawSampledWithRandom.filter(ee.Filter.geometry(testFC).Not())

    # Classifier to test
    classifier = ee.Classifier.smileRandomForest(
        numberOfTrees=500,
        variablesPerSplit=20,
        minLeafPopulation=1,
        bagFraction=0.632,
        maxNodes=33554432,
        seed = 0).setOutputMode('REGRESSION')
    
    # define the Train classifier
    trainedClassifer = classifier.train(trainFC, varToModel, covarsToUse)
    # Apply classifier to the feature collection
    classified = testFC.classify(classifier = trainedClassifer,
                                 outputName = 'predicted')
    # Get predicted value
    predicted = classified.first().get('predicted')
    # return the predicted value for each feature
    return f.set('predicted', predicted).copyProperties(f)

In [7]:
# Define the R^2 claculation function
def coefficientOfDetermination(fcOI,propertyOfInterest,propertyOfInterest_Predicted):
    # Compute the mean of the property of interest
    propertyOfInterestMean = ee.Number(ee.Dictionary(ee.FeatureCollection(fcOI).select([propertyOfInterest]).reduceColumns(ee.Reducer.mean(),[propertyOfInterest])).get('mean'));
    # Compute the total sum of squares
    def totalSoSFunction(f):
        return f.set('Difference_Squared',ee.Number(ee.Feature(f).get(propertyOfInterest)).subtract(propertyOfInterestMean).pow(ee.Number(2)))
    totalSumOfSquares = ee.Number(ee.Dictionary(ee.FeatureCollection(fcOI).map(totalSoSFunction).select(['Difference_Squared']).reduceColumns(ee.Reducer.sum(),['Difference_Squared'])).get('sum'))
    # Compute the residual sum of squares
    def residualSoSFunction(f):
        return f.set('Residual_Squared',ee.Number(ee.Feature(f).get(propertyOfInterest)).subtract(ee.Number(ee.Feature(f).get(propertyOfInterest_Predicted))).pow(ee.Number(2)))
    residualSumOfSquares = ee.Number(ee.Dictionary(ee.FeatureCollection(fcOI).map(residualSoSFunction).select(['Residual_Squared']).reduceColumns(ee.Reducer.sum(),['Residual_Squared'])).get('sum'))
    # Finalize the calculation
    r2 = ee.Number(1).subtract(residualSumOfSquares.divide(totalSumOfSquares))
    return ee.Number(r2)

In [8]:
#  define a feature collection to save the calcuation results
bloo_cv_fc = ee.FeatureCollection(ee.List(buffer_sizes).map(lambda n: ee.Feature(ee.Geometry.Point([0,0])).set('buffer_size',n)))
# define the R2 calc function 
def calc_final_r2(buffer_feat):
    # Add buffer to FC of sampled observations
    buffer = buffer_feat.get('buffer_size')

    fc_wBuffer = rawSampledWithRandom.map(lambda f: f.buffer(buffer))
    # Apply blocked leave one out CV function
    predicted = fc_wBuffer.map(BLOOcv)
    # Calculate R2 value
    R2_val = coefficientOfDetermination(predicted, varToModel, 'predicted')
    return(buffer_feat.set('R2_val', R2_val))

    ########################
    ## Uncomment the lines below to export the predicted/observed data per buffer size
    # predObs = predicted.select([varToModel, 'predicted'])
    # to_export = predObs.toList(50000).getInfo()
    # result = []
    # for item in to_export:
    #     values = item['properties']
    #     row = [str(values[key]) for key in [varToModel, 'predicted']]
    #     row = ",".join(row)
    #     result.append(row)
    #
    # df = pd.DataFrame([item.split(",") for item in result], columns = [varToModel, 'predicted'])
    # df['buffer_size'] = buffer
    # with open('temp/exported_df.csv', 'a') as f:
    #     df.to_csv(f, mode='a', header=f.tell()==0)

# Calculate R2 across range of R2 values
final_fc = bloo_cv_fc.map(calc_final_r2)

In [9]:
#  define a feature collection to save the calcuation results
bloo_cv_fc = ee.FeatureCollection(ee.List(buffer_sizes).map(lambda n: ee.Feature(ee.Geometry.Point([0,0])).set('buffer_size',n)))
# define the R2 calc function 
def calc_Pred_Obs(buffer_feat):
    # Add buffer to FC of sampled observations
    buffer = buffer_feat.get('buffer_size')

    fc_wBuffer = rawSampledWithRandom.map(lambda f: f.buffer(buffer))
    # Apply blocked leave one out CV function
    predicted = fc_wBuffer.map(BLOOcv)
    # Uncomment the lines below to export the predicted/observed data per buffer size
    predObs = predicted.select([varToModel, 'predicted'])
    return(predObs)
    
# Calculate predObs across range of R2 values
final_PredObs = bloo_cv_fc.map(calc_Pred_Obs)
# define a list to save the index value of the buffer sizes
indexSeries = range(0,len(buffer_sizes))
for idx in indexSeries:
    filteredData = final_PredObs.toList(10000).get(idx)
    predObs_export = ee.batch.Export.table.toAsset(
        collection = filteredData,
        description = varToModel+'bloo_cv_LeafHabit_'+str(buffer_sizes[idx])+'m',
        assetId = 'users/leonidmoore/LeafHabitProject/SpatialCV/Spatial_CV_of_LeafHabit_PredObs_'+str(buffer_sizes[idx])+'m')
    # start the exportation
    predObs_export.start()